In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import sys
from html.parser import HTMLParser
from html.entities import name2codepoint
sns.set(color_codes=True)
import warnings
warnings.filterwarnings("ignore")                   
import nltk                                         
nltk.download('stopwords')
from nltk.corpus import stopwords                   
from nltk.stem import PorterStemmer  
LA = np.linalg

from sklearn.feature_extraction.text import CountVectorizer          
from sklearn.feature_extraction.text import TfidfVectorizer          
from gensim.models.word2vec import Word2Vec                                  
import re
%matplotlib inline

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/chd415/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load training data text
def load_data(filename):
    load_file = pd.read_csv(filename,delimiter=',', header=0,
                        dtype={'name':str, 'lvl1':str, 'lvl2':str, 'lvl3':str, 'descrption':str, 'type':str})
    load_file.columns = ['id', 'name','lvl1','lvl2','lvl3','descrption','price','type']
    load_file.duplicated(subset=None, keep='first')
    load_file.set_index('id', inplace = True)
    load_file.head()
    return load_file
#print(len(train_file))
def load_label(filename):
    load_label = pd.read_csv(filename,delimiter=',', header=0)
    load_label.columns = ['id', 'score']
    load_label.duplicated(subset=None, keep='first')
    load_label.set_index('id', inplace = True)
    return load_label

In [3]:
def map_mathod(column):
    values = []
    indexs = []
    mapping = {}
    index = 0
    for count in range(len(train_file)):
        value = train_file.get_value(count+1,column)
        if value in values and value != np.nan:
            continue
        values.append(value)
        indexs.append(len(values))
    for j in range(len(indexs)):
        mapping[values[j]] = indexs[j]
    mapping[np.nan] = 0.0
    return mapping
#train_file['lvl3'] = train_file['lvl3'].str.lower().replace('[^\'\w]+',' ',regex=True)
#mapping_lvl3 = map_mathod('lvl3')
#print(mapping_lvl3)

In [4]:
def text_embedding(column):
    temp_X = column.astype(str)
    stop = set(stopwords.words('english'))
    temp =[]
    snow = nltk.stem.SnowballStemmer('english')

    for sentence in temp_X: 
        words = [snow.stem(word) for word in sentence.split(' ') if word not in stopwords.words('english')]   # Stemming and removing stopwords
        temp.append(sentence)

    count_vect = CountVectorizer(max_features=5000)
    bow_data = count_vect.fit_transform(temp)

    final_tf = temp
    tf_idf = TfidfVectorizer(max_features=5000)
    tf_data = tf_idf.fit_transform(final_tf)
    w2v_data = temp
    splitted = []
    for row in w2v_data: 
        splitted.append([word for word in row.split()])     #splitting words
    
    train_w2v = Word2Vec(splitted,min_count=1,size=50, workers=4)
    avg_data = []
    for row in splitted:
        vec = np.zeros(50, dtype=float)
        count = 0
        for word in row:
            try:
                vec += train_w2v[word]
                count += 1
            except:
                pass
        if (count == 0):
            avg_data.append(vec)
        else:
            avg_data.append(vec/count)
#        avg_data.append(vec)
    
    return avg_data

In [ ]:
#test cell
temp = load_data('train_data.csv')
temp['descrption'] = temp['descrption'].str.lower()
description_X = temp.descrption.str.lower().replace('<.*?>','',regex=True).replace('[^\w\s]+',' ',regex=True)


In [ ]:
#test cell
def test_embedding(column):
    temp_X = column.astype(str)
    stop = set(stopwords.words('english'))
    temp =[]
    snow = nltk.stem.SnowballStemmer('english')

    for sentence in temp_X: 
        words = [snow.stem(word) for word in sentence.split(' ') if word not in stopwords.words('english')]   # Stemming and removing stopwords
        temp.append(sentence)

    count_vect = CountVectorizer(max_features=5000)
    bow_data = count_vect.fit_transform(temp)

    final_tf = temp
    tf_idf = TfidfVectorizer(ngram_range=(1,2),stop_words = 'english',max_features=5000)
    tf_data = tf_idf.fit_transform(final_tf)
    w2v_data = temp
    splitted = []
    for row in w2v_data: 
        splitted.append([word for word in row.split()])     #splitting words
    
    train_w2v = Word2Vec(splitted,min_count=5,size=50, workers=4)
    avg_data = []
    for row in splitted:
        vec = np.zeros(50, dtype=float)
        count = 0
        for word in row:
            try:
                vec += train_w2v[word]
                count += 1
            except:
                pass
        if (count == 0):
            avg_data.append(vec)
        else:
            avg_data.append(vec/count)
#        avg_data.append(vec)

    tf_w_data = []
    tf_data = tf_data.toarray()
    i = 0
    for row in splitted:
        vec = [0 for i in range(50)]
    
        temp_tfidf = []
        for val in tf_data[i]:
            if val != 0:
                temp_tfidf.append(val)
    
        count = 0
        tf_idf_sum = 0
        for word in row:
            try:
                count += 1
                tf_idf_sum = tf_idf_sum + temp_tfidf[count-1]
                vec += (temp_tfidf[count-1] * train_w2v[word])
            except:
                pass
        if (tf_idf_sum == 0):
            tf_w_data.append(vec)
        else:
            tf_w_data.append(vec/tf_idf_sum)
#            vec = float(1/tf_idf_sum) * vec
#        tf_w_data.append(vec)
        i = i + 1
    
    return tf_w_data

In [ ]:
#test cell
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

temp['descrption'] = test_embedding(description_X)
temp.descrption.head()
#tfidf_n = TfidfVectorizer(ngram_range=(1,2),stop_words = 'english')

In [5]:
def clean_data(filename):
    #clean up data for lvl 1&2&3
    filename['lvl1'] = filename['lvl1'].str.lower().replace('[^a-zA-Z]+',' ',regex=True)
    filename['lvl2'] = filename['lvl2'].str.lower().replace('[^\'\w]+',' ',regex=True)
    filename['lvl3'] = filename['lvl3'].str.lower().replace('[^\'\w]+',' ',regex=True)
    filename['descrption'] = filename['descrption'].str.lower()
    filename['name'] = filename['name'].str.lower()
    
    mapping_lvl1 = map_mathod('lvl1')
    mapping_lvl2 = map_mathod('lvl2')
    mapping_lvl3 = map_mathod('lvl3')
    
    filename['lvl1'] = filename['lvl1'].map(mapping_lvl1)
    filename['lvl2'] = filename['lvl2'].map(mapping_lvl2)
    filename['lvl3'] = filename['lvl3'].map(mapping_lvl3)
    
    #normalize price
    maxp = filename.price.max()
    valuethred = 600.
    filename['price'] = filename['price'].clip(lower=0.,upper=valuethred).div(valuethred,fill_value=None)
    #hist = train_file['price'].hist(bins=10)
    #maxp

    #clean up type 
    mapping_type = {'international':1.,'local':2., np.nan:0.}
    filename['type'] = filename['type'].map(mapping_type)
    
    #clean up text
    description_X = filename.descrption.str.lower().replace('<.*?>','',regex=True).replace('[^\w\s]+',' ',regex=True)
    filename['descrption'] = text_embedding(description_X)
    
    name_X = filename.name.str.lower().replace('<.*?>','',regex=True).replace('[^\w\s]+',' ',regex=True)
    filename['name'] = text_embedding(name_X)

    return filename,mapping_lvl1,mapping_lvl2,mapping_lvl3

In [6]:
train_file = load_data('train_data.csv')
cleaned_train,mapping_lvl1,mapping_lvl2,mapping_lvl3 = clean_data(train_file)
train_score = load_label('train_label.csv')

In [ ]:
# test cell
cleaned_train['descrption'] = temp['descrption']

In [7]:
cleaned_train.head(10)
#print(mapping_lvl1)

,name,lvl1,lvl2,lvl3,descrption,price,type
id,,,,,,,
1,"[0.1112114392583155, -0.10216980137758785, 0.0...",1.0,1.0,1.0,"[0.08458858515627475, -0.05196527924595608, 0....",0.213333,1.0
2,"[-0.4643172339004065, 0.18790752374167954, 0.3...",2.0,2.0,2.0,"[-0.0821757186204195, 0.5118478735676035, 0.98...",0.024483,1.0
3,"[-0.01779988112817095, 0.9916763860224322, -0....",3.0,3.0,3.0,"[-0.5880512452567928, 0.6610294657875784, -0.2...",0.023500,1.0
4,"[-0.011420761180274627, 1.0104831246768726, -0...",3.0,3.0,3.0,"[0.09876202584912433, 0.5161218158293354, -0.4...",0.029900,1.0
5,"[-0.11495634096482878, 0.764323537272182, -0.2...",3.0,3.0,3.0,"[0.09029336935944027, 0.24826423823833466, -0....",0.011333,1.0
6,"[0.3112053220661787, -0.07709115697070956, 0.0...",4.0,4.0,4.0,"[0.014964479953050613, 0.05194679504515639, 0....",0.648317,1.0
7,"[-0.044303586648311466, 0.2982949146946125, -0...",2.0,5.0,5.0,"[-0.26340331617070156, 0.08268022217736062, 0....",1.000000,2.0
8,"[0.6431926132904159, 0.08405322457353274, -0.0...",5.0,6.0,6.0,"[-0.43231389040334356, -0.1330042877899749, -0...",0.036233,1.0
9,"[0.24886354403570293, 0.06856095297262073, 0.0...",6.0,7.0,7.0,"[-0.19626282444818222, -0.2013505833381985, 0....",0.041667,2.0


In [8]:
from sklearn import preprocessing

def rearrange(cleaned_data):
    la = cleaned_data.lvl1.as_matrix(columns=None).tolist()
    lb = cleaned_data.lvl2.as_matrix(columns=None).tolist()
    lc = cleaned_data.lvl3.as_matrix(columns=None).tolist()

    X = la
    X = np.column_stack((X,lb))
    X = np.column_stack((X,lc))

    enc = preprocessing.OneHotEncoder()
    enc.fit(X)
    X = enc.transform(X).toarray()
    
    ld = cleaned_data.price.as_matrix(columns=None).tolist()
    le = cleaned_data.type.as_matrix(columns=None).tolist()

    X = np.column_stack((X,ld))
    X = np.column_stack((X,le))

    lf = cleaned_data.name.as_matrix(columns=None).tolist()
    lg = cleaned_data.descrption.as_matrix(columns=None).tolist()
    lg_array = np.vstack( lg )
    lf_array = np.vstack( lf )

    U, s, Vh = LA.svd(lg_array, full_matrices=False)
    assert np.allclose(lg_array, np.dot(U, np.dot(np.diag(s), Vh)))

    s[8:] = 0.
    new_lg = np.dot(U, np.dot(np.diag(s), Vh))

    Uf, sf, Vhf = LA.svd(lf_array, full_matrices=False)
    assert np.allclose(lf_array, np.dot(Uf, np.dot(np.diag(sf), Vhf)))

    sf[5:] = 0.
    new_lf = np.dot(Uf, np.dot(np.diag(sf), Vhf))
    
    X = np.column_stack((X,new_lf))
    X = np.column_stack((X,new_lg))
    X = X.tolist()
    return X

    
    #print(len(X))
X = rearrange(cleaned_train)
Y = train_score.score.as_matrix(columns=None).tolist()

In [9]:
from sklearn.utils import shuffle
X, Y = shuffle(X, Y)
from sklearn.model_selection import train_test_split
X_train, X_validation, y_train, y_validation = train_test_split(X, Y, test_size=0.20, random_state=0)

from keras.preprocessing import sequence
maxlen = 150
X_train = sequence.pad_sequences(X_train, maxlen=maxlen, dtype='float32')
X_validation = sequence.pad_sequences(X_validation, maxlen=maxlen, dtype='float32')
#X_train = np.any(np.isnan(X_train))
#X_train = np.all(np.isfinite(X_train))
print(X_train[1400].size)

Using TensorFlow backend.


150


In [10]:
import os
import time
from sklearn import metrics
from sklearn import preprocessing
import numpy as np
import pandas as pd
import random
import math
import warnings
warnings.filterwarnings("ignore")

from matplotlib import pyplot as plt

from sklearn import preprocessing
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectPercentile
from sklearn.feature_selection import chi2

from sklearn import linear_model
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.ensemble import BaggingClassifier

from sklearn.model_selection import cross_val_score

In [11]:
# Linear Regression Classifier
def linear_regression_classifier(train_x, train_y):
    model = linear_model.LinearRegression()
    model.fit(train_x, train_y)
    return model
 
# Multinomial Naive Bayes Classifier
def naive_bayes_classifier(train_x, train_y):
    model = MultinomialNB()

    param_grid = {'alpha': [math.pow(10,-i) for i in range(11)]}
    grid_search = GridSearchCV(model, param_grid, n_jobs = 1, verbose=1)
    grid_search.fit(train_x, train_y)
    best_parameters = grid_search.best_estimator_.get_params()
    
    model = MultinomialNB(alpha = best_parameters['alpha'])  
    model.fit(train_x, train_y)
    return model
 
 
# KNN Classifier
def knn_classifier(train_x, train_y):
    model = KNeighborsClassifier()

    param_grid = {'n_neighbors': list(range(1,21))}
    grid_search = GridSearchCV(model, param_grid, n_jobs = 1, verbose=1)
    grid_search.fit(train_x, train_y)
    best_parameters = grid_search.best_estimator_.get_params()
    
    model = KNeighborsClassifier(n_neighbors = best_parameters['n_neighbors'], algorithm='kd_tree')

    bagging = BaggingClassifier(model, max_samples=0.5, max_features=1 )
    bagging.fit(train_x, train_y)
    return bagging
 
 
# Logistic Regression Classifier
def logistic_regression_classifier(train_x, train_y):
    model = LogisticRegression(penalty='l2')
    model.fit(train_x, train_y)
    return model
 
 
# Random Forest Classifier
def random_forest_classifier(train_x, train_y):
    model = RandomForestClassifier()

    param_grid = {'n_estimators': list(range(1,21))}
    grid_search = GridSearchCV(model, param_grid, n_jobs = 1, verbose=1)
    grid_search.fit(train_x, train_y)
    best_parameters = grid_search.best_estimator_.get_params()
    
    model = RandomForestClassifier(n_estimators = best_parameters['n_estimators'])
    
    model.fit(train_x, train_y)
    return model
 
 
# Decision Tree Classifier
def decision_tree_classifier(train_x, train_y):
    model = tree.DecisionTreeClassifier()
    model.fit(train_x, train_y)

    bagging = BaggingClassifier(model, max_samples=0.5, max_features=1 )
    bagging.fit(train_x, train_y)
    return bagging
 
 
# GBDT(Gradient Boosting Decision Tree) Classifier
def gradient_boosting_classifier(train_x, train_y):
    model = GradientBoostingClassifier()
    
    model = RandomForestClassifier()

    param_grid = {'n_estimators': list(range(100,300,10))}
    grid_search = GridSearchCV(model, param_grid, n_jobs = 1, verbose=1)
    grid_search.fit(train_x, train_y)
    best_parameters = grid_search.best_estimator_.get_params()
    
    model = RandomForestClassifier(n_estimators = best_parameters['n_estimators'])

    model.fit(train_x, train_y)
    return model

# SVM Classifier
def svm_classifier(train_x, train_y):
    model = SVC(kernel='linear', probability=True)
    model.fit(train_x, train_y)
    return model
 
# SVM Classifier using cross validation
def svm_cross_validation(train_x, train_y):
    model = SVC(kernel='linear', probability=True)
    param_grid = {'C': [1e-3, 1e-2, 1e-1, 1, 10, 100, 1000], 'gamma': [0.001, 0.0001]}
    grid_search = GridSearchCV(model, param_grid, n_jobs = 1, verbose=1)
    grid_search.fit(train_x, train_y)
    best_parameters = grid_search.best_estimator_.get_params()
    #for para, val in best_parameters.items():
        #print para, val
    model = SVC(kernel='rbf', C=best_parameters['C'], gamma=best_parameters['gamma'], probability=True)
    model.fit(train_x, train_y)
    return model

def feature_select(x,y):
    clf = ExtraTreesClassifier()
    clf = clf.fit(x, y)
    model = SelectFromModel(clf, prefit=True)
    return model

In [ ]:
# just for my own record


if __name__ == '__main__': 
    thresh = 0.5    
#    model_save_file = "/home/jason/datamining/model/models"     
#    model_save = {}
#    result_save_file = '/home/jason/datamining/result/results' 
     
    test_classifiers = ['KNN','LR','RF','DT']    
    classifiers = {
                   'KNN':knn_classifier,
                    'LR':logistic_regression_classifier,
                    'RF':random_forest_classifier,
                    'DT':decision_tree_classifier,                 
    }
        
    print('reading training and testing data...')    
    #X_train, X_validation, y_train, y_validation
    select_model = feature_select(X_train, y_train)
    X_train = select_model.transform(X_train)
    X_validation = select_model.transform(X_validation)

    result = []
        
    for classifier in test_classifiers:    
        print('******************* %s ********************' % classifier)    
        start_time = time.time()    
        model = classifiers[classifier](X_train, y_train)   
        print('training took %fs!' % (time.time() - start_time))    
        predict = model.predict(X_validation)

        precision = metrics.precision_score(y_validation, predict)    
        recall = metrics.recall_score(y_validation, predict)    
        print('precision: %.2f%%, recall: %.2f%%' % (100 * precision, 100 * recall))    
        accuracy = metrics.accuracy_score(y_validation, predict)    
        print('accuracy: %.2f%%' % (100 * accuracy))

        scores = cross_val_score(model, X_train, y_train)
        print(scores)

In [ ]:
#test cnn model

from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers import Conv1D, MaxPooling1D
from keras.datasets import imdb

# Embedding
max_features = 5000
maxlen = 150
embedding_size = 128

# Convolution
kernel_size = 5
filters = 64
pool_size = 4

# LSTM
lstm_output_size = 70

# Training
batch_size = 30
epochs = 2

'''
Note:
batch_size is highly sensitive.
Only 2 epochs are needed as the dataset is very small.
'''

print('Loading data...')
#X_train, X_validation, y_train, y_validation
#(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(X_train), 'train sequences')
print(len(X_validation), 'test sequences')

print('Pad sequences (samples x time)')
#x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
#x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', X_train.shape)
print('x_test shape:', X_validation.shape)

print('Build model...')

model = Sequential()
model.add(Embedding(max_features, embedding_size, input_length=maxlen))
#model.add(Dense(32, activation='relu', input_dim=100))
model.add(Dropout(0.25))
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))
model.add(MaxPooling1D(pool_size=pool_size))
model.add(LSTM(lstm_output_size))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(X_validation, y_validation))
score, acc = model.evaluate(X_validation, y_validation, batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

In [12]:
test_file = load_data('test_data.csv')
#cleaned_test = clean_data(test_file)
X_train, y_train = X,Y

In [13]:
filename = test_file
#clean up data for lvl 1&2&3
filename['lvl1'] = filename['lvl1'].str.lower().replace('[^a-zA-Z]+',' ',regex=True)
filename['lvl2'] = filename['lvl2'].str.lower().replace('[^\'\w]+',' ',regex=True)
filename['lvl3'] = filename['lvl3'].str.lower().replace('[^\'\w]+',' ',regex=True)
filename['descrption'] = filename['descrption'].str.lower()
filename['name'] = filename['name'].str.lower()
    
#    mapping_lvl1 = map_mathod('lvl1')
#    mapping_lvl2 = map_mathod('lvl2')
#    mapping_lvl3 = map_mathod('lvl3')
    
filename['lvl1'] = filename['lvl1'].map(mapping_lvl1)
filename['lvl2'] = filename['lvl2'].map(mapping_lvl2)
filename['lvl3'] = filename['lvl3'].map(mapping_lvl3)
    
    #normalize price
maxp = filename.price.max()
valuethred = 600.
filename['price'] = filename['price'].clip(lower=0.,upper=valuethred).div(valuethred,fill_value=None)
    #hist = train_file['price'].hist(bins=10)
    #maxp

    #clean up type 
mapping_type = {'international':1.,'local':2., np.nan:0.}
filename['type'] = filename['type'].map(mapping_type)
    
    #clean up text
description_X = filename.descrption.str.lower().replace('<.*?>','',regex=True).replace('[^\w\s]+',' ',regex=True)
filename['descrption'] = text_embedding(description_X)
    
name_X = filename.name.str.lower().replace('<.*?>','',regex=True).replace('[^\w\s]+',' ',regex=True)
filename['name'] = text_embedding(name_X)

#    return filename

In [14]:
cleaned_test = filename
lat = cleaned_test.lvl1.as_matrix(columns=None).tolist()
lbt = cleaned_test.lvl2.as_matrix(columns=None).tolist()
lct = cleaned_test.lvl3.as_matrix(columns=None).tolist()

Xt = lat
Xt = np.column_stack((Xt,lbt))
Xt = np.column_stack((Xt,lct))

enct = preprocessing.OneHotEncoder()
enct.fit(Xt)
Xt = enct.transform(Xt).toarray()

ldt = cleaned_test.price.as_matrix(columns=None).tolist()
let = cleaned_test.type.as_matrix(columns=None).tolist()

Xt = np.column_stack((Xt,ldt))
Xt = np.column_stack((Xt,let))

lft = cleaned_test.name.as_matrix(columns=None).tolist()
lgt = cleaned_test.descrption.as_matrix(columns=None).tolist()
lg_arrayt = np.vstack( lgt )
lf_arrayt = np.vstack( lft )

Ut, st, Vht = LA.svd(lg_arrayt, full_matrices=False)
assert np.allclose(lg_arrayt, np.dot(Ut, np.dot(np.diag(st), Vht)))

st[10:] = 0.
new_lgt = np.dot(Ut, np.dot(np.diag(st), Vht))

Uf, sf, Vhf = LA.svd(lf_arrayt, full_matrices=False)
assert np.allclose(lf_arrayt, np.dot(Uf, np.dot(np.diag(sf), Vhf)))

sf[5:] = 0.
new_lft = np.dot(Uf, np.dot(np.diag(sf), Vhf))


#X = la
#X = np.column_stack((X,lb))
#X = np.column_stack((X,lc))
#X = np.column_stack((X,ld))
#X = np.column_stack((X,le))
Xt = np.column_stack((Xt,new_lft))
Xt = np.column_stack((Xt,new_lgt))
Xt = Xt.tolist()
#cleaned_test = Xt
#return X,s

In [15]:
#cleaned_test = clean_test(test_file)
#cleaned_test = filename
cleaned_test.head(10)

,name,lvl1,lvl2,lvl3,descrption,price,type
id,,,,,,,
18142,"[0.2910505139401981, 0.04427307163963893, 0.00...",5.0,21.0,166.0,"[0.4974043408189626, 0.03650149753358325, 0.22...",0.081667,2.0
18143,"[-0.0008019954548217356, 0.2919247818063013, 0...",9.0,32.0,47.0,"[-0.5066361925659739, 0.24731269746790735, 0.8...",0.005383,1.0
18144,"[0.032295664971960444, 0.647023093678789, 0.00...",9.0,32.0,68.0,"[-0.3520956976743251, 0.24306542413850205, 0.6...",0.041783,1.0
18145,"[0.17624174249875876, 0.02362049308915933, 0.0...",1.0,25.0,40.0,"[-0.26399251909460875, -0.02353048401128035, 0...",0.196667,2.0
18146,"[0.27825695250390303, 0.4065230344939563, -0.0...",1.0,47.0,106.0,"[0.419692053925246, 0.010667627832541863, 0.21...",0.191333,1.0
18147,"[-0.08950607349666265, 0.5613701820660096, 0.2...",2.0,52.0,181.0,"[-0.144322803347475, -0.11823871918022633, 0.8...",0.013333,1.0
18148,"[-0.0010690220321218173, 0.9767497105834385, -...",2.0,2.0,84.0,"[0.1553927365069588, 0.5552089061765444, 1.336...",0.028950,1.0
18149,"[0.22926618655522665, 0.044200843976189695, -0...",1.0,1.0,1.0,"[-0.24068953905953094, 0.07620786027982832, 0....",0.017333,1.0
18150,"[0.14598204439076093, 0.3737279163100399, 0.09...",2.0,19.0,177.0,"[0.04258049875497818, 0.9404868662357331, 2.60...",0.057550,1.0


In [16]:
#from keras.preprocessing import sequence
#maxlen = 150
X_test = rearrange(cleaned_test)
#X_train = sequence.pad_sequences(X_train, maxlen=maxlen, dtype='float32')
#X_test = sequence.pad_sequences(X_test, maxlen=maxlen, dtype='float32')

In [18]:
if __name__ == '__main__': 
    thresh = 0.5    
#    model_save_file = "/home/jason/datamining/model/models"     
#    model_save = {}
#    result_save_file = '/home/jason/datamining/result/results' 
     
    test_classifiers = ['LR','RF']    
    classifiers = {
                   'LR':logistic_regression_classifier,
                   'RF':random_forest_classifier         
    }
        
    print('reading training and testing data...')    
    #X_train, X_validation, y_train, y_validation
#    X_test = rearrange(Xt)
#    X_test = Xt
    select_model = feature_select(X_train, y_train)
    X_train = select_model.transform(X_train)
    X_test = select_model.transform(X_test)

    result = []
    
    start_time = time.time()
    model = classifiers['LR'](X_train, y_train)   
    print('training took %fs!' % (time.time() - start_time))    
    Y_predict_lr = model.predict_proba(X_test)[:,1]
    print('predict finished')
    
    model = classifiers['RF'](X_train, y_train)   
    print('training took %fs!' % (time.time() - start_time))    
    Y_predict_rf = model.predict_proba(X_test)[:,1]
    print('predict finished')
    
    Y_predict = []
    for i in range(len(Y_predict_rf)):
        Y_predict.append((Y_predict_lr[i] + Y_predict_rf[i]) / 2.0)
        
#    for classifier in test_classifiers:    
#        print('******************* %s ********************' % classifier)    
#        start_time = time.time()    
#        model = classifiers[classifier](X_train, y_train)   
#        print('training took %fs!' % (time.time() - start_time))    
#        Y_predict = model.predict_proba(X_test)[:,1]
#        print('predict finished')

reading training and testing data...
training took 0.186404s!
predict finished
Fitting 3 folds for each of 20 candidates, totalling 60 fits


[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:   30.4s finished


training took 33.509690s!
predict finished


In [21]:
Y_predict

[0.8753618815616444,
 0.558157957130386,
 0.7346397451325362,
 0.7982709246250408,
 0.7529775888185125,
 0.7086165942435747,
 0.6804558254034009,
 0.6014131820887256,
 0.6652423409275128,
 0.6613623412731372,
 0.8246220680117915,
 0.5764272009459317,
 0.6230980745846861,
 0.6070260755547547,
 0.8047708291157444,
 0.34770011105394566,
 0.8514530020137331,
 0.5993519697899268,
 0.6922362632819707,
 0.6108998325067976,
 0.6892892224826154,
 0.7999942644408942,
 0.6915838272017485,
 0.7143226026828122,
 0.8376872702329736,
 0.7807174541017853,
 0.7266510912656121,
 0.8139099078963132,
 0.765441215789599,
 0.728271397619115,
 0.5955901480379577,
 0.7595465622689227,
 0.8227441110128091,
 0.5779241489665796,
 0.5118353720042099,
 0.43855214586064306,
 0.6971970931101545,
 0.7138950492347378,
 0.54285696358531,
 0.8229581545573905,
 0.5699621170457707,
 0.6636332263306659,
 0.8302194771837841,
 0.6763691782162033,
 0.6251543309966527,
 0.8120451729462754,
 0.6685703431075003,
 0.3962351304266

In [24]:
temp_score = load_label('submission.csv')
submit_score  = temp_score
submit_score['score'] = Y_predict
submit_score.to_csv('predict_result.csv')

In [ ]:
table = pd.pivot_table(train_file,values = 'price', index=['lvl1','lvl2','lvl3'],columns=['type'],aggfunc=[min, max, np.mean])
table